## Ejercicios de pair programming 11 de enero: ETL 3.1

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! 

En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. 

Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, 

el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.


📌 Nota Todo lo tendremos que hacer desde jupyter notebook


Cread la BBDD con el nombre de tiburones.

Cread las tablas de la BBDD:

Tabla ataques

Tabla clima

BONUS Insertar los datos en las tablas.

📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [1]:
#importamos las librerias
import pandas as pd
import mysql.connector


In [2]:
#abrimos el archivo de tiburones limpio
df_tiburon = pd.read_csv('../datos/tiburon8.csv', index_col = 0)
df_tiburon.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0


In [3]:
#abrimos el archivo del clima
df_clima = pd.read_csv('../datos/meteo_tiburon.csv', index_col= 0)
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [4]:
#hacemos una funcion para crear la base de datos
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab"
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [5]:
#llamamos a la funcion y creamos la base de datos 'tiburones'
mybd= crear_bbdd('tiburones')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [6]:
#creamos una funcion para crear tabla nueva
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [7]:
#hacemos la query para la tabla ataques
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `case_number` VARCHAR(20) NOT NULL ,
  `year`INT NOT NULL,
  `type` VARCHAR(50) NOT NULL,
  `country` VARCHAR(50),
  `activity` VARCHAR (200),
  `injury` TEXT,
  `mes` VARCHAR (10),
  `deceso` VARCHAR(20),
  `genero` VARCHAR (20),
  `especies` VARCHAR (50),
  `edad` FLOAT,
  PRIMARY KEY (`case_number`))
ENGINE = InnoDB;
'''

In [8]:
df_clima.reset_index(inplace= True)   #generamos una columna de index para usarla como primary key en la tabla de clima

In [26]:
#hacemos la query para la tabla clima

tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `index` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT,
  `cloudcover` INT, 
  `highcloud` INT, 
  `midcloud` INT, 
  `lowcloud` INT,
  `rh_profile` TEXT, 
  `wind_profile` TEXT, 
  `temp2m` INT, 
  `lifted_index` INT, 
  `rh2m` INT,
  `msl_pressure` INT, 
  `prec_type` VARCHAR (10), 
  `prec_amount` INT, 
  `snow_depth` INT,
  `wind10m_direction` INT, 
  `wind10m_speed` INT, 
  `pais` VARCHAR (20),
  PRIMARY KEY (`index`))
ENGINE = InnoDB;
'''

In [10]:
#llamamos a la funcion
crear_insertar_tabla('tiburones', "AlumnaAdalab", tabla_ataques)

In [27]:
#llamamos a la funcion
crear_insertar_tabla('tiburones', "AlumnaAdalab", tabla_clima)

Ya tenemos generada la base de datos con las dos columnas, ataques y clima

Introducimos ahora los datos en las tablas correspondientes

In [12]:
#hemos visto que en las columnas injury y activity los registros estan  un poco sucios ya que empiezan con ' y terminan por ", asi que hacemos una funcion para igualarlas y poder hacer la insercion bien
def cambiar_comillas(col):
    return col.replace("'", "").replace("\"", "")


In [13]:
df_clima.head()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA
3,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 270, 'speed':...",14,15,1,1028,none,0,0,60,2,USA
4,4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",14,15,3,1027,none,0,0,85,2,USA


In [14]:
df_tiburon["injury"] = df_tiburon["injury"].apply(cambiar_comillas)

In [15]:
df_tiburon["activity"]=df_tiburon["activity"].apply(cambiar_comillas)

In [16]:
for indice, fila in df_tiburon.iterrows(): # itreamos por el dataframe.
    
    
    query_tiburon = f"""
            INSERT INTO ataques (case_number, year, type, country, activity, injury, mes, deceso, genero, especies, edad) 
            VALUES ( "{fila['case_number']}", "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['injury']}", "{fila['mes']}", "{fila['deceso']}", "{fila['genero']}", "{fila['especies']}", "{fila['edad']}");
            """
    crear_insertar_tabla("tiburones", "AlumnaAdalab", query_tiburon)
print("Ya estan los datos insertados en tu tabla")

Ya estan los datos insertados en tu tabla


In [28]:
for indice, fila in df_clima.iterrows(): # itreamos por el dataframe.
       
    query_clima = f"""
            INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile, wind_profile, temp2m, lifted_index, rh2m, msl_pressure, prec_type, 
            prec_amount, snow_depth, wind10m_direction, wind10m_speed, pais) 
            VALUES ("{fila['timepoint']}", "{fila['cloudcover']}", "{fila['highcloud']}", "{fila['midcloud']}", "{fila['lowcloud']}", "{fila['rh_profile']}", "{fila['wind_profile']}", "{fila['temp2m']}", "{fila['lifted_index']}", "{fila['rh2m']}", 
            "{fila['msl_pressure']}", "{fila['prec_type']}", "{fila['prec_amount']}", "{fila['snow_depth']}", "{fila['wind10m.direction']}", "{fila['wind10m.speed']}", "{fila['pais']}");
            """
    crear_insertar_tabla("tiburones", "AlumnaAdalab", query_clima)
print("Ya estan los datos insertados en tu tabla")

Ya estan los datos insertados en tu tabla
